In [2]:
from pydantic import Field
from deepeval.dataset import Golden
from deepeval.test_case import LLMTestCase

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from benchmarq.utility import Evaluator
from benchmarq.experiment import Experiment

from dotenv import load_dotenv

load_dotenv()

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# noinspection PyPep8Naming
def generate(prompt: str) -> str:
    return generator(prompt)


class Test(Evaluator):
    def __init__(self):
        super().__init__()

    def evaluate_consumption(self, input: Golden):
        generate(input.input)

    def evaluate_test_case(self, input: Golden) -> LLMTestCase:
        output = generate(f"{input.context}: {input.input}")[0]["generated_text"]
        print(output)
        return LLMTestCase(input=input.input, expected_output=input.expected_output, actual_output=output, context=input.context, retrieval_context=input.retrieval_context)

experiment = Experiment(
    subquestion_id="subquestionId",
    subquestion_path="experiments/test/tests.json",
    name="name",
    description="A very long description",
    settings=Test())

print(experiment.run())


Device set to use mps:0
[codecarbon INFO @ 10:15:57] [setup] RAM Tracking...
[codecarbon INFO @ 10:15:57] [setup] GPU Tracking...
[codecarbon INFO @ 10:15:57] No GPU found.
[codecarbon INFO @ 10:15:57] [setup] CPU Tracking...
[codecarbon WARNING @ 10:15:57] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon INFO @ 10:16:01] CPU Model on constant consumption mode: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
[codecarbon INFO @ 10:16:01] >>> Tracker's metadata:
[codecarbon INFO @ 10:16:01]   Platform system: macOS-15.3.1-x86_64-i386-64bit
[codecarbon INFO @ 10:16:01]   Python version: 3.11.0
[codecarbon INFO @ 10:16:01]   CodeCarbon version: 2.2.2
[c

[]: Something went down that I don't know what but it's not my fault. I'm still at fault here and now I'm stuck, and the others are in their minds. The man in the mask had to be brought in, and


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[]: Lucy-Sue "The Devil is here", at 614 N. High Street, Toronto, ON M2S 2Y2

Awards for the 50th anniversary of the original publication to be held in the library of Canada


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 4 test case(s) in parallel: |██████████|100% (4/4) [Time Taken: 00:07,  1.97s/test case]



Metrics Summary

  - ❌ Correctness (GEval) (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The actual output contains entirely different content than the expected output, with no alignment of data points or consistency in conclusions. It mentions a location and an event, while the expected output is a continuation of a famous phrase., error: None)
  - ❌ Succinctness (GEval) (score: 0.009720499348103148, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The Actual Output contains two sentences and two ending punctuation marks, not fulfilling the requirement of a single sentence., error: None)

For test case:

  - input: Lucy
  - actual output: []: Lucy-Sue "The Devil is here", at 614 N. High Street, Toronto, ON M2S 2Y2

Awards for the 50th anniversary of the original publication to be held in the library of Canada
  - expected output: in the sky with diamonds.
  - context: []
  - retrieval context: []


Metrics Summary

  - ❌ Correctness (G

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

RunResult(consumption_results=ConsumptionResult(timestamp='2025-03-04T10:16:28', project_name='codecarbon', run_id='1a89f511-40a6-4386-b057-1a64bb8c0e8c', experiment_id='1', duration=16.82456398010254, emissions=4.4102781621935665e-05, emissions_rate=2.621332812790485e-06, cpu_power=22.5, gpu_power=0.0, ram_power=6.0, cpu_energy=0.00010514379888772964, gpu_energy=0.0, ram_energy=2.8035672903060914e-05, energy_consumed=0.00013317947179079055, country_name='The Netherlands', country_iso_code='NLD', region='north brabant', cloud_provider='', cloud_region='', os='macOS-15.3.1-x86_64-i386-64bit', python_version='3.11.0', codecarbon_version='2.2.2', cpu_count=16.0, cpu_model='Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz', gpu_count=None, gpu_model=None, longitude=5.0661, latitude=51.5542, ram_total_size=16.0, tracking_mode='machine', on_cloud='N', pue=1), metric_results=[TestResult(name='test_case_1', success=False, metrics_data=[MetricData(name='Correctness (GEval)', threshold=0.5, success=Fals

In [7]:
import os
import pandas as pd

a=pd.read_csv("/Users/leenlaptop/Documents/repos/greenai/green.ai/experiments/test/inputs.csv")

print(a)

       input            expected_output  \
0  Something      in the way she moves.   
1       Lucy  in the sky with diamonds.   

                                             context  retrieval_context  \
0  finish the sentence of the song of the beatles...                NaN   
1  finish the sentence of the song of the beatles...                NaN   

   tools_called  expected_tools  
0           NaN             NaN  
1           NaN             NaN  
